In [79]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import sys
import codecs
import csv
import requests
import ssl
from bs4 import BeautifulSoup
from datetime import datetime


# get image url from thumbnail
# Return text "NO IMAGE" if no image
def getImageURL(block):
    imageTagData = block.select_one('div.cassetteitem_object-item img[src]')
    if imageTagData.has_attr('rel'):
        return imageTagData['rel']
    else:
        return "NO IMAGE"

# get all items without Class
# もしクラス違いがあったりするとタグだけで取れないとき(getAllWithTagClass)で,
# クラス("")空白にすることはできるがクラスがあるやつを取得しなくなる
def getAllWithTag(source, tag):
    return source.find_all(tag)

# get all items with Class
def getAllWithTagClass(source, tag, className):
    return source.find_all(tag, attrs={'class':className})

# get text with div tag
def getTextWithTag(tag, block, className):
    return block.find(tag, attrs={'class':className}).text

# if two divs are under certain item with div tag
def getTextWithTag2(tag, block):
    data = block.find_all(tag)
    return data[0].text + "," + data[1].text

def getTextWithTag3(tag, block):
    data = block.find_all(tag)
    return data[0].text + "," + data[1].text + "," + data[2].text

# get item with li tag
def getItemWithLi(block, className):
    return block.find('li', attrs={'class': className})

# to retrieve each data
# aptPicture = 画像URL = データx1
# aptName = 物件名 = データx1
# aptAddress = 住所 = データx1
# aptTransportation = 交通手段 = データx3
# aptDetail = 築年数/階数 = データx2
###
def generateBasicData(item):
    aptPicture, aptName, aptAddress = "", "", ""
    aptTransportation, aptDetail, comma = "", "", ","
    aptPicture = getImageURL(item) # get aptPicture
    aptName = getTextWithTag('div', item, 'cassetteitem_content-title') # get aptName
    aptAddress = getTextWithTag('li', item, 'cassetteitem_detail-col1') # get aptAddress
    cassetteitem_detail_col2s = getItemWithLi(item, 'cassetteitem_detail-col2') # get a block for aptTransportation
    aptTransportation = getTextWithTag3('div', cassetteitem_detail_col2s) # get aptTransportation
    cassetteitem_detail_col3s = getItemWithLi(item, 'cassetteitem_detail-col3') # get a block for aptDetail
    aptDetail = getTextWithTag2('div', cassetteitem_detail_col3s) # get aptDetail
    #print(aptName)
    return aptPicture + comma + aptName + comma + aptAddress + comma + aptTransportation + comma + aptDetail


# to retrieve each data
# [0]- [1]- [2]階 Floor [3]賃料 Rent [4]管理費 Admin [5]敷/礼/保証/敷引,償却 Sec 
# [6]間取り type [7]専有面積 square [8]- [9]お気に入り [10]- Link
# 
###
def generateRoomData(item):
    rmData = ""
    tds = getAllWithTag(item, 'td')
    for i in range(2,8):
        rmData += (tds[i].text + ",") 
    rmData += tds[10].find('a')['href'] # get Link(詳細ページ用)
    return rmData


# Generating data
#
###
def generateData(items, tmpDataList):
    basicDataInfo = ""
    for item in items:
        basicDataInfo = generateBasicData(item)
        rooms = getAllWithTag(item, 'tbody')
        for room in rooms:
            roomDataInfo = generateRoomData(room)
            tmpDataList.append(basicDataInfo + "," + roomDataInfo)
    return tmpDataList

# to get pagenation
# return True if thre is next page else False(ページネーション)
def checkPagenation(source):
    pagenation = source.find('div', attrs={'class' : 'pagination pagination_set-nav'})
    backOrForward = pagenation.find_all("p")
    right = False
    for bof in backOrForward:
        bofText = bof.find('a').text
        if bofText == "次へ":
            right = True
    return right

def main():
    # set initial page number
    page = 1
    url = "http://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=040&bs=040&ta=17&sc=17206&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&pn={page}"
    dataList = ["写真,名前,住所,移動1,移動2,移動3,築年数,全階数,部屋階,賃料,管理費,敷/礼/保証/敷引|償却,間取り,専有面積,詳細リンク"] # to store each csv data
    fileName = "suumo"
    with requests.Session() as session: # create session
        while True:
            response = session.get(url.format(page=page))
            # store contents in soup
            soup = BeautifulSoup(response.content, "html.parser")

            # Parent box which contain all the data for each apt
            items = getAllWithTagClass(soup, 'div', 'cassetteitem')            
            
            
            dataList = generateData(items, dataList) # update 
            
            # break if there is no more "Next Page"
            break
            if not checkPagenation(soup):
                break

            page += 1

    with open(fileName + '.csv', 'w') as f:
        wr = csv.writer(f, delimiter='\n')
        wr.writerow(dataList)
    #print(dataList)
    print("SUCCESS")
    
if __name__ == '__main__':
    main()

SUCCESS


In [ ]:

    imageTagData = block.select_one('div.cassetteitem_object-item img[src]')
    if 'rel' in imageTagData:
        print("YES")
    else:
        print("FUCK")
    return ""